In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

import torch
import torchvision.transforms as transforms
from models.crnn import init_network
from datasets.datahelpers import default_loader
from utils.converter import LabelConverter

In [3]:
# load alphabet from file
alphabet = ''
with open('alphabet/alphabet_decode_5990.txt', mode='r', encoding='utf-8') as f:
    for line in f.readlines():
        alphabet += line.strip()

In [60]:
img_name = 'out/target.jpg'
device = torch.device("cpu")
model_path = 'pretrained/params_6.pth'
# densenet121_pretrained.pth
model_params = {}
model_params['architecture'] = "densenet121"
model_params['num_classes'] = len(alphabet) + 1
model_params['mean'] = (0.5,)
model_params['std'] = (0.5,)
model = init_network(model_params)
model = model.to(device)

# load checkpoint
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint)

converter = LabelConverter(alphabet, ignore_case=False)

transform = transforms.Compose([
#     transforms.Resize((32, 280)),
    transforms.ToTensor(),
#     transforms.Normalize(mean=model.meta['mean'], std=model.meta['std']),
])

print('image name: {}'.format(img_name))
img = default_loader(img_name)

model_path2 = 'pretrained/densenet121_pretrained.pth'
# 
model_params2 = {}
model_params2['architecture'] = "densenet121"
model_params2['num_classes'] = len(alphabet) + 1
model_params2['mean'] = (0.5,)
model_params2['std'] = (0.5,)
model_2 = init_network(model_params2)
model_2 = model_2.to(device)

# load checkpoint
checkpoint2 = torch.load(model_path2, map_location='cpu')
model_2.load_state_dict(checkpoint2)


image name: out/target.jpg


In [61]:
import numpy as np
# plt.imshow(img)
img = img.convert("L")
img = np.asarray(img)
img = torch.Tensor(img).unsqueeze(0)
# img = img.
img = img.permute(1,2,0)
img.shape

torch.Size([32, 237, 1])

In [6]:
# transform
img = transform(img)
img = img.unsqueeze(0)
img = img.to(device)

In [7]:
with torch.no_grad():
    log_probs = model(img)
    preds_strs = converter.best_path_decode(log_probs, strings=True).decode('utf-8')
    preds = converter.best_path_decode(log_probs, raw=True).decode('utf-8')
    print('pred: {}'.format(preds))
    print('pred strings: {}'.format(preds_strs))

    log_probs2 = model_2(img)
    preds_strs2 = converter.best_path_decode(log_probs2, strings=True).decode('utf-8')
    preds2 = converter.best_path_decode(log_probs2, raw=True).decode('utf-8')

    print('pred2: {}'.format(preds2))
    print('pred2 strings: {}'.format(preds_strs2))

pred: ----二--三-映---互-宗--化--N--九--十-
pred strings: 二三映互宗化N九十
pred2: -一--幺--三-惦---互-六--化--八--九--十-
pred2 strings: 一幺三惦互六化八九十


AttributeError: 'Tensor' object has no attribute 'convert'

In [4]:
import torch
a = torch.Tensor([[0,1],[2,3]])

In [5]:
a.permute(1,0) 

tensor([[0., 2.],
        [1., 3.]])